In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-5cutcntw
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-5cutcntw
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=7835405e28f8bce248575ac00cf0ae37885e2179bb17e3b51f7299d332d9a903
  Stored in directory: /tmp/pip-ephem-wheel-cache-2e_v3201/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%writefile add.cu
#include <iostream>
#include <cstdlib> // Include <cstdlib> for rand()
using namespace std;
__global__
void add(int* A, int* B, int* C, int size) {
int tid = blockIdx.x * blockDim.x + threadIdx.x;
if (tid < size) {
C[tid] = A[tid] + B[tid];
}
}
void initialize(int* vector, int size) {
for (int i = 0; i < size; i++) {
vector[i] = rand() % 10;
}
}
void print(int* vector, int size) {
for (int i = 0; i < size; i++) {
cout << vector[i] << " ";
}

}
cout << endl;
}
int main() {
int N = 4;
int* A, * B, * C;
int vectorSize = N;
size_t vectorBytes = vectorSize * sizeof(int);
// Allocate host memory
A = new int[vectorSize];
B = new int[vectorSize];
C = new int[vectorSize];
// Initialize host arrays
initialize(A, vectorSize);
initialize(B, vectorSize);
cout << "Vector A: ";
print(A, N);
cout << "Vector B: ";
print(B, N);
int* X, * Y, * Z;
// Allocate device memory
cudaMalloc(&X, vectorBytes);
cudaMalloc(&Y, vectorBytes);
cudaMalloc(&Z, vectorBytes);
// Check for CUDA memory allocation errors
if (X == nullptr || Y == nullptr || Z == nullptr) {
cerr << "CUDA memory allocation failed" << endl;
return 1;
}
// Copy data from host to device
cudaMemcpy(X, A, vectorBytes, cudaMemcpyHostToDevice);
cudaMemcpy(Y, B, vectorBytes, cudaMemcpyHostToDevice);
int threadsPerBlock = 256;
int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;
// Launch kernel
add<<<blocksPerGrid, threadsPerBlock>>>(X, Y, Z, N);
// Check for kernel launch errors
cudaError_t kernelLaunchError = cudaGetLastError();
if (kernelLaunchError != cudaSuccess) {
cerr << "CUDA kernel launch failed: " << cudaGetErrorString(kernelLaunchError) return 1;
}
// Copy result from device to host
cudaMemcpy(C, Z, vectorBytes, cudaMemcpyDeviceToHost);
Practical All.ipynb - Colab https://colab.research.google.com/drive/1Du6Y-gMe...

// Check for CUDA memcpy errors
cudaError_t memcpyError = cudaGetLastError();
if (memcpyError != cudaSuccess) {
cerr << "CUDA memcpy failed: " << cudaGetErrorString(memcpyError) << endl;
return 1;
}
cout << "Addition: ";
print(C, N);
// Free device memory
cudaFree(X);
cudaFree(Y);
cudaFree(Z);
// Free host memory
delete[] A;
delete[] B;
delete[] C;
return 0;
}
Writing add.cu

Writing add.cu


In [ ]:
!nvcc add.cu -o add
!./add
Vector A: 3 6 7 5
Vector B: 3 5 6 2

SyntaxError: invalid syntax (<ipython-input-1-2d486d831e67>, line 3)